<a href="https://colab.research.google.com/github/gKorada/MLPractice/blob/main/Crime_Consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.8 MB/s eta 0:00:00


In [ ]:
# Stop old session if it exists
import os
import json
import time
from confluent_kafka import Consumer, KafkaException
from collections import defaultdict


In [ ]:
KAFKA_BOOTSTRAP_SERVERS = "lkc-07zjp5.us-east-2.cloud.aws.confluent.cloud:9092"
KAFKA_API_KEY = "LAGPJCXEU7IYUOIU"
KAFKA_API_SECRET = "0Uv2wo401gnXikKTyoQCS/ZOUw/NjiuORe7RbsItqhpFKJVU6WNqjN3Qkfa4VW5z"
TOPIC = "CrimeStreaming"

In [ ]:
conf = {
    'bootstrap.servers': KAFKA_BOOTSTRAP_SERVERS,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_API_KEY,
    'sasl.password': KAFKA_API_SECRET,
    'group.id': 'crime-consumer-group-1',
    'auto.offset.reset': 'latest'
}

# --- Actionable mapping ---
recommendations = {
    "BURGLARY": "🚨 Increase residential patrols",
    "THEFT OF IDENTITY": "🛡️ Launch fraud awareness campaign",
    "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT": "🧠 Deploy crisis intervention teams",
    "THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD": "📢 Encourage neighborhood watch",
}

# --- Initialize Consumer ---
c = Consumer(conf)
c.subscribe([TOPIC])

crime_counts = defaultdict(int)
THRESHOLD = 3  # Trigger alert if any crime appears more than this

print("🔍 Listening for crime reports...\n")

try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())

        # Parse JSON message
        data = json.loads(msg.value().decode('utf-8'))

        # Extract key fields
        crime_type = data.get("Crm_Cd_Desc", "UNKNOWN")
        area = data.get("AREA_NAME", "UNKNOWN")
        timestamp = data.get("DATE_OCC", "UNKNOWN")

        # Update count
        crime_counts[crime_type] += 1

        # Display
        print(f"🕒 {timestamp} | 📍 {area} | 🚔 Crime: {crime_type}")
        print(f"   Count so far: {crime_counts[crime_type]}")

        # Trigger action if needed
        if crime_counts[crime_type] == THRESHOLD:
            action = recommendations.get(crime_type, "📌 Monitor closely and review local response plans.")
            print(f"⚠️ ALERT: Spike in {crime_type.upper()} — {action}\n")

except KeyboardInterrupt:
    print("\n🔌 Stopping consumer...")
finally:
    c.close()


🔍 Listening for crime reports...


🔌 Stopping consumer...
